## make data for history endpoints

In [1]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import json
import numpy as np

In [2]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)
Base.prepare(engine, reflect=True)
session = Session(engine)

In [3]:
#create dataframe
brew_df = pd.read_sql_query("SELECT * FROM business", con=engine)
brew_df

,id,CompanyName,street,City,State,ZipCode,ZipPlus,ContactFirstName,ContactLastName,ContactTitle,...,BusinessType,County,Employee,AnnualSales,URL,YearEstablished,latitude,longitude,google_rating,google_numratings
0,0,217 Brew Works,217 South St S,Wilson,NC,27893,4911.0,Tom,Curran,Owner,...,Private,Wilson,14,10453000.0,217BREWWORKS.COM,2016,35.722939,-77.912148,4.6,83.0
1,1,All Sevens Brewing,777 Haywood Rd,Asheville,NC,28806,3132.0,None,None,None,...,Private,Buncombe,23,34511000.0,None,2018,35.578687,-82.594280,4.8,4.0
2,2,Altamont Brewing Co,1050 Haywood Rd,Asheville,NC,28806,2639.0,None,None,None,...,Private,Buncombe,23,34511000.0,None,2017,35.581549,-82.604400,4.3,169.0
3,3,Andrews Brewing Co,1026 Main St,Andrews,NC,28901,NaN,None,None,None,...,Private,Cherokee,14,17916000.0,None,2018,35.200849,-83.824356,4.7,146.0
4,4,Angry Troll Brewing,222 E Main St # U2,Elkin,NC,28621,3493.0,Cicely,Mc Culloch,Owner,...,Private,Surry,12,16648000.0,ANGRYTROLLBREWING.COM,2015,36.244335,-80.848763,4.3,281.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,271,Wise Man Brewing,826 Angelos Bros Ave,Winston Salem,NC,27101,1524.0,Michael D,Beverly,Manager,...,Private,Forsyth,14,5870000.0,WISEMANBREWING.COM,2017,36.104924,-80.245300,4.7,302.0
272,272,Wooden Robot Brewery LLC,1440 S Tryon St # 110,Charlotte,NC,28203,4263.0,Joshua,Patton,CEO,...,Private,Mecklenburg,14,7049000.0,WOODENROBOTBREWERY.COM,2015,35.216754,-80.856676,4.6,530.0
273,273,Yesteryears Brewery,300 E Main St # C,Carrboro,NC,27510,2359.0,David,Larsen,Manager,...,Private,Orange,14,21819000.0,None,2016,35.910023,-79.068814,4.6,88.0
274,274,York Chester Brewing Co,1500 River Dr # 250,Belmont,NC,28012,3587.0,None,None,None,...,Private,Gaston,14,20830000.0,YORKCHESTERBREW.COM,2017,35.236705,-81.015616,4.9,56.0


In [4]:
# create database of breweries founded per year
brew_df.loc[brew_df['YearEstablished'] <= 1990, 'YearEstablished'] = 1990
yeargrouped = pd.DataFrame(brew_df.groupby('YearEstablished').count()['CompanyName']).reset_index()
yeargrouped = yeargrouped.rename(columns={'YearEstablished':'yearEstablished', 'CompanyName':'yearCount'})

In [5]:
# make shell with all years
yearShell = pd.DataFrame({'yearEstablished':[y for y in range(1990,2020)]})
yearFinal = pd.merge(yearShell,yeargrouped,on='yearEstablished',how='left').fillna(0)
yearFinal['cumulativeCount'] = yearFinal['yearCount'].cumsum()
yearFinal

,yearEstablished,yearCount,cumulativeCount
0,1990,4.0,4.0
1,1991,1.0,5.0
2,1992,0.0,5.0
3,1993,0.0,5.0
4,1994,0.0,5.0
5,1995,1.0,6.0
6,1996,2.0,8.0
7,1997,2.0,10.0
8,1998,0.0,10.0
9,1999,2.0,12.0


In [6]:
# add data to sqlite database
yearFinal.to_sql('history', con=engine, if_exists='replace')